## Initializing Environment

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_tools').getOrCreate()

23/08/21 23:36:44 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/21 23:36:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 23:36:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Tokenization

In [4]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [5]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [3]:
sentence_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,Regression,models,are,neat')
], ['id', 'sentence'])

In [4]:
sentence_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,Regressi...|
+---+--------------------+



In [9]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [11]:
count_tokens = udf(lambda words:len(words), IntegerType())  ## create a user defined function

In [12]:
tokenized = tokenizer.transform(sentence_df)

In [13]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,Regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [14]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [15]:
rg_tokenized = regex_tokenizer.transform(sentence_df)

In [16]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,Regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



## Stop Words Removal

In [17]:
from pyspark.ml.feature import StopWordsRemover

In [18]:
sentence_tokenized_df = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
],['id','tokens'])

In [19]:
sentence_tokenized_df.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [20]:
remover = StopWordsRemover(inputCol='tokens', outputCol='stop_words_removed')

In [21]:
remover.transform(sentence_tokenized_df).show()

+---+--------------------+--------------------+
| id|              tokens|  stop_words_removed|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



## N-Grams

In [3]:
from pyspark.ml.feature import NGram

In [10]:
wordDataFrame = spark.createDataFrame([
    (0, ['Hi','I','heard','about','Spark']),
    (1, ['I','wish','Java','could','use','case','classes']),
    (2, ['Logistic','Regression','models','are','neat'])
], ['id', 'words'])

In [11]:
nGram = NGram(n=2, inputCol='words', outputCol='grams')

In [12]:
nGram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic Regression, Regression models, models are, are neat]    |
+------------------------------------------------------------------+



## Term Frequency and Inverse Document Frequency

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [6]:
sentence_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,Regression,models,are,neat')
], ['label', 'sentence'])

In [7]:
sentence_df.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I heard about ...|
|    1|I wish Java could...|
|    2|Logistic,Regressi...|
+-----+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [9]:
words_data = tokenizer.transform(sentence_df)

In [10]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|
|    1|I wish Java could...|[i, wish, java, c...|
|    2|Logistic,Regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [11]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)

In [12]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)
idf_data = idf_model.transform(featurized_data)

In [13]:
idf_data.show()

23/08/21 23:48:51 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/08/21 23:48:53 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|(262144,[18700,19...|
|    1|I wish Java could...|[i, wish, java, c...|(262144,[19036,20...|(262144,[19036,20...|
|    2|Logistic,Regressi...|[logistic,regress...|(262144,[11534],[...|(262144,[11534],[...|
+-----+--------------------+--------------------+--------------------+--------------------+



## Count Vectorizer

In [14]:
from pyspark.ml.feature import CountVectorizer

In [15]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ['id', 'words'])

In [16]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [17]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2)

In [18]:
cv_model = cv.fit(df)

In [20]:
results = cv_model.transform(df)

In [22]:
results.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

